In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sympy as sy
from IPython.core.interactiveshell import InteractiveShell
import datetime

InteractiveShell.ast_node_interactivity = "all"

sy.init_printing()

%matplotlib inline

/home/accrist/anaconda3/envs/modeling/lib/python3.12/site-packages/numpy/_core/getlimits.py:548: UserWarning: Signature b'\x00\xd0\xcc\xcc\xcc\xcc\xcc\xcc\xfb\xbf\x00\x00\x00\x00\x00\x00' for <class 'numpy.longdouble'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)


## proof of concept: perturb and stablize system

### theory

In [2]:
G_h, G, I, ISF, k_h, t = sy.symbols("G_h G I ISF k_h t")

In [3]:
sy.Eq(sy.Derivative(G, t), k_h * t * (G_h - G))

### Implementation

In [4]:
T = pd.date_range(start='2025-05-01', end='2025-05-02', freq="1min")
T

DatetimeIndex(['2025-05-01 00:00:00', '2025-05-01 00:01:00',
               '2025-05-01 00:02:00', '2025-05-01 00:03:00',
               '2025-05-01 00:04:00', '2025-05-01 00:05:00',
               '2025-05-01 00:06:00', '2025-05-01 00:07:00',
               '2025-05-01 00:08:00', '2025-05-01 00:09:00',
               ...
               '2025-05-01 23:51:00', '2025-05-01 23:52:00',
               '2025-05-01 23:53:00', '2025-05-01 23:54:00',
               '2025-05-01 23:55:00', '2025-05-01 23:56:00',
               '2025-05-01 23:57:00', '2025-05-01 23:58:00',
               '2025-05-01 23:59:00', '2025-05-02 00:00:00'],
              dtype='datetime64[ns]', length=1441, freq='min')

In [5]:
T[0]

Timestamp('2025-05-01 00:00:00')

In [6]:
stopper = 0

for i in T:
    print(i, type(i))
    print(i-T[0])
    if stopper > 5:
        break
    stopper += 1

2025-05-01 00:00:00 <class 'pandas._libs.tslibs.timestamps.Timestamp'>
0 days 00:00:00
2025-05-01 00:01:00 <class 'pandas._libs.tslibs.timestamps.Timestamp'>
0 days 00:01:00
2025-05-01 00:02:00 <class 'pandas._libs.tslibs.timestamps.Timestamp'>
0 days 00:02:00
2025-05-01 00:03:00 <class 'pandas._libs.tslibs.timestamps.Timestamp'>
0 days 00:03:00
2025-05-01 00:04:00 <class 'pandas._libs.tslibs.timestamps.Timestamp'>
0 days 00:04:00
2025-05-01 00:05:00 <class 'pandas._libs.tslibs.timestamps.Timestamp'>
0 days 00:05:00
2025-05-01 00:06:00 <class 'pandas._libs.tslibs.timestamps.Timestamp'>
0 days 00:06:00


In [13]:
G_i = 60  # initial glucose
G_homeostatic_target = 100
k_glucose_return = 0.001 # "normal"; gets 60 -> 100 or 140 -> 100 over 60 minutes
t_step = 60  # 60s = 1 min

G_log = list()

dG_log = list()



for t_i in T:
    print(G_i)
    G_log.append(G_i)
    dG_log.append(k_glucose_return * t_step * (G_homeostatic_target - G_i))
    G_i += k_glucose_return * t_step * (G_homeostatic_target - G_i)

60
62.4
64.65599999999999
66.77663999999999
68.77004159999998
70.64383910399998
72.40520875775998
74.06089623229438
75.61724245835671
77.08020791085531
78.45539543620399
79.74807171003175
80.96318740742984
82.10539616298405
83.179072393205
84.1883280496127
85.13702836663595
86.02880666463778
86.86707826475951
87.65505356887394
88.39575035474151
89.09200533345702
89.7464850134496
90.36169591264263
90.93999415788407
91.48359450841103
91.99457883790637
92.47490410763199
92.92640986117406
93.35082526950362
93.7497757533334
94.1247892081334
94.4773018556454
94.80866374430667
95.12014391964827
95.41293528446937
95.68815916740121
95.94686961735714
96.19005744031571
96.41865399389677
96.63353475426297
96.83552266900719
97.02539130886676
97.20386783033476
97.37163576051468
97.5293376148838
97.67757735799077
97.81692271651133
97.94790735352065
98.0710329123094
98.18677093757084
98.29556468131659
98.39783080043759
98.49396095241133
98.58432329526664
98.66926389755065
98.7491080636976
98.824161579

In [16]:
pd.DataFrame({"glucose":G_log, "dG":dG_log}, index=T)

,glucose,dG
2025-05-01 00:00:00,60.000000,2.400000e+00
2025-05-01 00:01:00,62.400000,2.256000e+00
2025-05-01 00:02:00,64.656000,2.120640e+00
2025-05-01 00:03:00,66.776640,1.993402e+00
2025-05-01 00:04:00,68.770042,1.873798e+00
...,...,...
2025-05-01 23:56:00,100.000000,6.821210e-15
2025-05-01 23:57:00,100.000000,6.821210e-15
2025-05-01 23:58:00,100.000000,6.821210e-15
2025-05-01 23:59:00,100.000000,6.821210e-15
